In [1]:
include("gates.jl")
using Compat.Test
import QuCircuit: PrimitiveBlock, CompositeBlock, MatrixBlock

In [11]:
GATES = [:X, :Y, :Z]
slower(s::Symbol) = s |> string |> lowercast |> Symbol
for G in GATES
    @eval struct $(Symbol(G, :Gate)){MT} <: PrimitiveBlock{1, MT} end
end

In [3]:
############ General Single Qubit Multi-Control Gate ###########
struct ControlBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    target::GT
    cbits::Vector{Int}
    ibit::Int
end

struct SingleControlBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    target::GT
    cbits::Vector{Int}
    ibit::Int
end

########### Control XYZ #############
abstract type ControlXYZ{N, MT} <: CompositeBlock{N, MT} end
for G in GATES
    @eval struct $Symbol(:C, G, :Gate){N, MT} <: ControlXYZ{N, MT}
        cbit::Int
        ibit::Int
    end
end

blocks(cb::ControlBlock) = [rb.target]

blocks (generic function with 1 method)

In [5]:
function mat(cb::ControlBlock{MatrixBlock, N, MT}) where {N, MT}
   controlled_U1(N, mat(MT, cb.target, ), cb.cbits, cb.ibit)
end

for (GATETYPE, MATFUNC) in zip(GATETYPES, [:cxgate, :cygate, :czgate])
    @eval function mat(cb::$GATETYPE{N, MT}) where {N, MT}
        $MATFUNC(MT, N, cb.cbit, cb.ibit)
    end
end

In [6]:
#function apply!(r::Register, c::RepeatedBlock{N, MT, XGate{MT}}) where {N, MT}
#end
cb = CXGate{2, ComplexF64}(2,1)

CXGate{2,Complex{Float64}}(2, 1)

In [7]:
@test mat(cb) == CNOT_MAT

Test Passed


In [18]:
mutable struct RepeatedBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    unit::GT
end
for (G, MATFUNC) in zip(GATES, [:xgate, :ygate, :zgate])
    GGate = Symbol(G, :Gate)
    GBlock = Symbol(G, :Block)
    @eval struct $GBlock{GT<:$GGate, N, MT} <: PrimitiveBlock{N, MT}
        target::GT
        bits::Vector{Int}
    end
    @eval function mat(cb::$GATETYPE{N, MT}) where {N, MT}
        $MATFUNC(MT, N, cb.ibit)
    end
end
blocks(rb::RepeatedBlock) = [rb.unit]

blocks (generic function with 2 methods)

In [ ]:
function mat(rb::RepeatedBlock{N, XGate{MT}}) where {N, MT}
    rb
end

for (GATETYPE, MATFUNC) in zip([:XGate, :ZGate], [:cnotgate, :czgate])
    @eval function mat(cb::RepeatedBlock{$GATETYPE, N, MT}) where {N, MT}
        $MATFUNC(N, cb.cbit, cb.ibit)
    end
end